In [ ]:
import pandas as pd
import datacompy
from buckaroo import BuckarooWidget
from datacompy_app import DatacompyBuckaroo

In [ ]:
# Create sample DataFrames
df_a = pd.DataFrame({
    'a': [2, 3, 4, 5, 6, 7, 8],
    'b': [5, 4, 9, 4, 6, 7, 8],
    'c': [ 'foo', 'bar', None, None, 'bar', 'bar', 'foo']})

df_b = pd.DataFrame({
    'a': [2, 3, 4, 5, 6, 7, 8],
    'b': [ 5, 4, 4, 4, 6, 4, 4],
    'd': ['baz', 'baz', 'bar', None, None, 'bar', 'bar'],
    'f': [10, 1, 200, 150, 140, 130, 120]
})  # Notice the difference in the last row

# a doesn't line up 
df_c = pd.DataFrame({
    'a': [3, 4, 5, 6, 7, 8, 9],
    'b': [4, 9, 7, 4, 4, 6, 4],
    'd': ['foo', 'baz', 'baz', None, None, 'bar', 'bar'],
})  # Notice the difference in the last row
DatacompyBuckaroo(df_a, df_c, join_columns=['a'], how='outer')

In [ ]:
# a doesn't line up 
df_d = pd.DataFrame({
    'a': [3, 4, 5, 6, 7, 8, 9],
    'b': [4, 9, 7, 4, 4, 6, 4],
    'c': [4, 9, 7, 4, 4, 6, 4],
})
# a doesn't line up 
df_e = pd.DataFrame({
    'a': [3, 4,     5, 60, 70, 80, 90],
    'b': [4, 9, 7,  4, 4, 6, 4],
    'c': [4.0, 9.1, None, 4, 4, 6, 4]
})
DatacompyBuckaroo(df_d, df_e, join_columns=['a'], how='outer')